In [1]:
# pip install names

In [2]:
import random
import time
from datetime import date
from dateutil.relativedelta import relativedelta
import pandas as pd
import names
import numpy as np
import math

In [3]:
sample_size = 50000

In [4]:
def str_time_prop(start, end, time_format, prop):
    """
    Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formatted in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, time_format))
    etime = time.mktime(time.strptime(end, time_format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(time_format, time.localtime(ptime))


def random_date(start, end):
    return str_time_prop(start, end, '%Y-%m-%d', random.random())

def get_expiry_date(old_date):
    year = int(old_date[:4])
    month = int(old_date[5:7])
    day = int(old_date[8:10])
    return date(year, month, day) + relativedelta(years=5)

def get_birth_date(old_date):
    year = int(old_date[:4])
    month = int(old_date[5:7])
    day = int(old_date[8:10])
    
    start = date(year, month, day) - relativedelta(years=100)
    end = date(year, month, day) - relativedelta(years=18)
    
    return str_time_prop(str(start), str(end), '%Y-%m-%d', random.random())

In [5]:
def data_generator(sample_size):
#     name = []
    date_of_birth = []
    opening_date = []
    expiry_date = []
    limit = []
    balance = []
    available_credit = []
    for i in range(sample_size):
#         name.append(names.get_full_name())
        opening_date.append(random_date('2000-01-01', '2022-12-31'))
        expiry_date.append(get_expiry_date(opening_date[i]))
        
        date_of_birth.append(get_birth_date(opening_date[i]))
        
        limit.append(float(random.randrange(500, 100001, 500)))
        balance.append(round(random.uniform(0, limit[i]), 2))
        available_credit.append(limit[i] - balance[i])
#     data = pd.DataFrame(list(zip(name, date_of_birth, opening_date, expiry_date, limit, balance, available_credit)),
#                         columns = ['Name', 'Date_of_birth', 'Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'])
    data = pd.DataFrame(list(zip(date_of_birth, opening_date, expiry_date, limit, balance, available_credit)),
                        columns = ['Date_of_birth', 'Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit'])
    return data

In [6]:
data = data_generator(sample_size)

In [7]:
data['Opening_date'] = data['Opening_date'].astype(str)
data['Expiry_date'] = data['Expiry_date'].astype(str)

In [8]:
data['Opening'] = data['Opening_date'].apply(lambda x: x.replace('-', ''))
data['Expiry'] = data['Expiry_date'].apply(lambda x: x.replace('-', ''))
data['Client_id'] = data['Opening'] + data['Expiry']
data['Client_id'] = data['Client_id'].apply(lambda x: int(x))
data['Opening'] = data['Opening'].apply(lambda x: int(x))
data['Expiry'] = data['Expiry'].apply(lambda x: int(x))

data = data[['Client_id', 'Date_of_birth', 'Opening_date', 'Expiry_date', 'Credit_limit', 'Statement_balance', 'Available_credit', 'Opening', 'Expiry']]

In [9]:
data.dtypes

Client_id              int64
Date_of_birth         object
Opening_date          object
Expiry_date           object
Credit_limit         float64
Statement_balance    float64
Available_credit     float64
Opening                int64
Expiry                 int64
dtype: object

In [10]:
data.head()

,Client_id,Date_of_birth,Opening_date,Expiry_date,Credit_limit,Statement_balance,Available_credit,Opening,Expiry
0,2010102320151023,1937-02-02,2010-10-23,2015-10-23,82500.0,49668.48,32831.52,20101023,20151023
1,2006030920110309,1950-10-14,2006-03-09,2011-03-09,40500.0,26136.40,14363.60,20060309,20110309
2,2006012920110129,1961-05-20,2006-01-29,2011-01-29,20500.0,8911.67,11588.33,20060129,20110129
3,2001080220060802,1940-08-16,2001-08-02,2006-08-02,55000.0,17091.77,37908.23,20010802,20060802
4,2016082420210824,1957-04-28,2016-08-24,2021-08-24,20500.0,11822.77,8677.23,20160824,20210824


In [11]:
data.to_csv('data_50000.csv', index=False)